In [2]:
import pandas as pd


In [13]:
# original info data
vih_data = pd.read_csv("../data/HIV_escape_mutations.csv",sep="\t")
vih_data.head()

,Mutation_type,Chain,Mutation,log10(Exptal),Exptal,mCSM-AB_Pred
0,ORIGINAL,G,A54E,1.995635,98.0,-0.013
1,ORIGINAL,G,A62M,1.919078,82.0,-0.761
2,ORIGINAL,G,A64K,1.949390,88.0,-0.167
3,ORIGINAL,G,A69E,2.004321,100.0,-0.610
4,ORIGINAL,G,A79W,2.025306,105.0,-0.246


In [14]:
vih_original = vih_data.loc[vih_data["Mutation_type"]=="ORIGINAL"].copy()
vih_reverse = vih_data.loc[vih_data["Mutation_type"]=="REVERSE"]
# reeplace original mutation nomenclature
vih_original['Mutation'] =vih_reverse["Mutation"].values
#sort values to appedn to prediction data table
vih_original.loc[:,"mut_code"] = (vih_reverse["Chain"]+vih_reverse["Mutation"].str[1:]).values
# vih_original.sort_values(by='mut_code',inplace=True,ascending=False)


In [21]:
with open('../data/mutant_list_VRC01.txt','w') as f:
    
    mutants = vih_original['Mutation']
    chains = vih_original['Chain']
        
    raw_text = (chains+'\t'+mutants).str.cat(sep='\n')
    f.write(raw_text)

## Then, run prediction with ABPRED

# ABPRED results analysis

In [15]:
vih_original.loc[(vih_original['Exptal'] < 33 ),"mutation-effect"] = "decreased-binding"
vih_original.loc[(vih_original['Exptal'] > 300 ),"mutation-effect"] = "increased-binding"
vih_original.loc[(vih_original['Exptal'] < 300 )&(vih_original['Exptal'] > 33 ),"mutation-effect"] = "neutral"

In [39]:
inc_muts = vih_original.loc[vih_original["mutation-effect"]=="increased-binding"]['mut_code'].values.tolist()
dec_muts = vih_original.loc[vih_original["mutation-effect"]=="decreased-binding"]['mut_code'].values.tolist()


In [40]:
dec_muts

['G245A', 'G331A', 'G332A', 'G334A', 'G335A', 'G417A', 'G427A', 'G434A']

In [20]:
abpred_results = pd.read_csv('../benchmark/VRC01_bench/mutations_results.csv',index_col=0)
abpred_results.reset_index(inplace=True)

In [25]:
abpred_results['mut_code'] = abpred_results['index'].str.split('.',expand=True)[1]

In [41]:
abpred_results.loc[abpred_results['mut_code'].isin(inc_muts)]

,index,ddg,mut_code
21,VRC01.G294K.Repair2.clean.mut.pdb,-0.142702,G294K
58,VRC01.G421A.Repair2.clean.mut.pdb,0.170120,G421A


In [42]:
abpred_results.loc[abpred_results['mut_code'].isin(dec_muts)]

,index,ddg,mut_code
13,VRC01.G245A.Repair2.clean.mut.pdb,0.588782,G245A
30,VRC01.G331A.Repair2.clean.mut.pdb,0.045580,G331A
31,VRC01.G332A.Repair2.clean.mut.pdb,1.315657,G332A
33,VRC01.G334A.Repair2.clean.mut.pdb,0.482500,G334A
34,VRC01.G335A.Repair2.clean.mut.pdb,0.851635,G335A
55,VRC01.G417A.Repair2.clean.mut.pdb,0.625398,G417A
60,VRC01.G427A.Repair2.clean.mut.pdb,0.224734,G427A
65,VRC01.G434A.Repair2.clean.mut.pdb,1.117073,G434A


In [29]:
abpred_results

,index,ddg,mut_code
0,VRC01.G163A.Repair2.clean.mut.pdb,0.085829,G163A
1,VRC01.G163K.Repair2.clean.mut.pdb,0.377447,G163K
2,VRC01.G163T.Repair2.clean.mut.pdb,-0.032543,G163T
3,VRC01.G165A.Repair2.clean.mut.pdb,0.262043,G165A
4,VRC01.G168A.Repair2.clean.mut.pdb,0.361899,G168A
...,...,...,...
73,VRC01.G69A.Repair2.clean.mut.pdb,0.230455,G69A
74,VRC01.G79A.Repair2.clean.mut.pdb,0.420649,G79A
75,VRC01.G87A.Repair2.clean.mut.pdb,0.248139,G87A
76,VRC01.G88A.Repair2.clean.mut.pdb,0.234766,G88A
